In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

from pathlib import Path

from utils.helper import find_project_root

In [2]:
DATASET_PATH = find_project_root() / Path("datasets/nlp/")

df = pd.read_csv(DATASET_PATH / "IMDB Dataset.csv.zip")

In [3]:
from nlp.data.preprocessing import TextPreprocessor
from nlp.features.embeddings import GensimWord2Vec, SentenceEmbedder, PretrainedSentenceEmbedder

In [4]:
df["sentiment"] = df["sentiment"].map({"positive": 1, "negative": 0})

In [5]:
preprocessor = TextPreprocessor()
df["review"] = preprocessor.preprocess_batch(df["review"])

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df["review"], df["sentiment"], test_size=0.2, random_state=42
)

X_train_tokens = preprocessor.tokenize_batch(X_train)
X_test_tokens = preprocessor.tokenize_batch(X_test)

In [7]:
w2v = GensimWord2Vec(workers=8)
w2v.train(X_train_tokens)

In [8]:
w2v.model.wv.most_similar("awesome")

[('amazing', 0.8170696496963501),
 ('excellent', 0.7961263060569763),
 ('fantastic', 0.7636290788650513),
 ('incredible', 0.7623229622840881),
 ('outstanding', 0.7200294733047485),
 ('cool', 0.7124485373497009),
 ('great', 0.686339259147644),
 ('brilliant', 0.6769931316375732),
 ('spellbinding', 0.6664395332336426),
 ('awful', 0.6644167304039001)]

# Encode sentences using the trained Word2Vec model

In [9]:
embedder = SentenceEmbedder(w2v)

X_train_embeddings = embedder.encode_corpus(X_train_tokens)
X_test_embeddings = embedder.encode_corpus(X_test_tokens)

In [10]:
X_train_embeddings.shape


(40000, 100)

# Model Training

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_embeddings, y_train)

y_pred = clf.predict(X_test_embeddings)

In [23]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.87      0.87      4961
           1       0.87      0.87      0.87      5039

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



In [13]:
confusion_matrix(y_test, y_pred)

array([[4296,  665],
       [ 662, 4377]])

In [14]:
mis_idx = np.where(y_pred != y_test)[0]

len(mis_idx)

1327

In [15]:
for i in mis_idx[:10]:
    print("\n\n----------")
    print("TEXT:", X_test.iloc[i])
    print("TRUE:", y_test.iloc[i])
    print("PRED:", y_pred[i])



----------
TEXT: i really liked this summerslam due to the look of the arena the curtains and just the look overall was interesting to me for some reason anyways this could have been one of the best summerslam s ever if the wwf didn t have lex luger in the main event against yokozuna now for it s time it was ok to have a huge fat man vs a strong man but i m glad times have changed it was a terrible main event just like every match luger is in is terrible other matches on the card were razor ramon vs ted dibiase steiner brothers vs heavenly bodies shawn michaels vs curt hening this was the event where shawn named his big monster of a body guard diesel irs vs 1 2 3 kid bret hart first takes on doink then takes on jerry lawler and stuff with the harts and lawler was always very interesting then ludvig borga destroyed marty jannetty undertaker took on giant gonzalez in another terrible match the smoking gunns and tatanka took on bam bam bigelow and the headshrinkers and yokozuna defended

# Using pre-trained Word2Vec embeddings

In [16]:
import gensim.downloader as api

glove = api.load("glove-wiki-gigaword-100")

embedder_glove = PretrainedSentenceEmbedder(glove)

X_train_glove = embedder_glove.encode_corpus(X_train_tokens)
X_test_glove = embedder_glove.encode_corpus(X_test_tokens)

[==================================================] 100.0% 128.1/128.1MB downloaded


In [17]:
clf_glove = LogisticRegression(max_iter=1000)
clf_glove.fit(X_train_glove, y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [18]:
y_pred = clf_glove.predict(X_test_glove)

In [20]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.81      0.80      4961
           1       0.81      0.80      0.80      5039

    accuracy                           0.80     10000
   macro avg       0.80      0.80      0.80     10000
weighted avg       0.80      0.80      0.80     10000



Pretrained embeddings did not outperform locally trained embeddings, likely due to domain mismatch and task misalignment.